![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

Text here

## Data Understanding

Text here

In [1]:
import os
# Need to run to make latest TF version work with TF hub
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
from transformers import pipeline
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split

c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
path = kagglehub.dataset_download("nileshmalode1/samsum-dataset-text-summarization")

print("Filepath to dataset: ", path)

In [ ]:
df_samsum_train = pd.read_csv("Data/samsum-train.csv")
df_samsum_test = pd.read_csv("Data/samsum-test.csv")
df_samsum_val = pd.read_csv("Data/samsum-validation.csv")

## Data Preparation
Text here

## Analysis

Text here

## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here